In [14]:
import pandas as pd
#import ACS data
data = pd.read_csv("data/census/usa_00005.csv")

# Ensure data types are consistent to avoid issues with replacement
data = data.apply(pd.to_numeric, errors='ignore')

# Replace known missing value codes (e.g., 999, 9999999) with NaN
data = data.replace({999: None, 9999999: None})

# Verify the column names in the dataset
print("Column Names:", data.columns)

# Check for column presence before applying any mappings
if 'SEX' in data.columns:
    # Mapping for SEX
    data['SEX'] = data['SEX'].replace({1: 'Male', 2: 'Female'})
else:
    print("Column 'SEX' not found in the dataset.")

# Check and map race
if 'RACHSING' in data.columns:
        race_mapping = {
            1: 'White', 
            2: 'Black/African American', 
            3: 'Other', 
            4: 'Asian/Pacific Islander', 
            5: 'Hispanic/Latino'
        }
        data['RACHSING'] = data['RACHSING'].map(race_mapping)
else:
    print("Column 'RACHSING' not found in the dataset.")

# Handle AGE binning
if 'AGE' in data.columns:
    bins = [0, 17, 24, 29, 39, 49, 64, float('inf')]
    labels = ['0-17', '18-24', '25-29', '30-39', '40-49', '50-64', '65+']
    data['AGE_GROUP'] = pd.cut(data['AGE'], bins=bins, labels=labels, right=True, include_lowest=True)

else:
    print("Column 'AGE' not found in the dataset.")
    
# Handle education mapping
if 'EDUC' in data.columns:
    edu_mapping = {
        0: 'Not a high school graduate',
        1: 'Not a high school graduate',
        2: 'Not a high school graduate',
        3: 'Not a high school graduate',
        4: 'Not a high school graduate',
        5: 'Not a high school graduate',
        6: 'High school graduate',
        7: 'Some college',
        8: 'Some college',
        9: 'Some college',
        10: 'College graduate',
        11: 'Postgraduate education',
        999: 'Missing' 
    }
    data['EDUC'] = data['EDUC'].map(edu_mapping)
else:
    print("Column 'EDUC' not found in the dataset.")

# Replace NaN with a placeholder for income or drop missing rows
if 'FTOTINC' in data.columns:
    data['FTOTINC'] = data['FTOTINC'].fillna(-1)  # Use -1 for missing income

    # Adjust bins to account for the placeholder if using -1
    bins = [-float('inf'), 0, 15000, 30000, 50000, 75000, 100000, 150000, 200000, float('inf')]
    labels = [
        'Missing', 'Under $15,000', '$15,000-30,000', '$30,000-50,000',
        '$50,000-75,000', '$75,000-100,000', '$100,000-150,000',
        '$150,000-200,000', 'Over $200,000'
    ]

    # Create income brackets
    data['INCOME_BRACKET'] = pd.cut(data['FTOTINC'], bins=bins, labels=labels)
else:
    print("Column 'FTOTINC' not found in the dataset.")

# Handle veteran status
if 'VETSTAT' in data.columns:
    vet_mapping = {
        0: 'Not a veteran',
        1: 'Not a veteran',
        2: 'Veteran',
    }
    data['VETSTAT'] = data['VETSTAT'].map(vet_mapping)
else:
    print("Column 'VETSTAT' not found in the dataset.")

# Summarize and save the cleaned data

print("Missing Values:", data.isnull().sum())
print("Unique Values:", data.nunique())
data.to_csv("cleaned_usa_00003.csv", index=False)
print("Data cleaning complete. File saved as 'cleaned_usa_00003.csv'.")

df = pd.read_csv('cleaned_usa_00003.csv')
df = df[df['AGE_GROUP'] != '0-17']
df.to_csv("votingagepopulation.csv", index=False)
print("Data cleaning complete. File saved as votingagepopulation.csv'.")

# Load the dataset
df = pd.read_csv('votingagepopulation.csv')

# Ensure 'PERWT' is treated as a numeric column
df['PERWT'] = pd.to_numeric(df['PERWT'], errors='coerce')

# Calculate state and sex totals
state_sex_totals = df.groupby(['STATEICP', 'SEX'])['PERWT'].sum().unstack(fill_value=0).reset_index()
state_sex_totals['Total'] = state_sex_totals.sum(axis=1)
state_sex_totals['Female_Percentage'] = (state_sex_totals.get('Female', 0) / state_sex_totals['Total']) 
state_sex_totals['Male_Percentage'] = (state_sex_totals.get('Male', 0) / state_sex_totals['Total']) 

# Calculate state and race totals
state_race_totals = df.groupby(['STATEICP', 'RACHSING'])['PERWT'].sum().unstack(fill_value=0).reset_index()
state_race_totals['Total'] = state_race_totals.sum(axis=1)
state_race_totals['White_Percentage'] = (state_race_totals.get('White', 0) / state_race_totals['Total']) 
state_race_totals['Black_Percentage'] = (state_race_totals.get('Black/African American', 0) / state_race_totals['Total']) 
state_race_totals['Hispanic_Percentage'] = (state_race_totals.get('Hispanic/Latino', 0) / state_race_totals['Total']) 
state_race_totals['Asian_Percentage'] = (state_race_totals.get('Asian/Pacific Islander', 0) / state_race_totals['Total']) 
state_race_totals['Other_Percentage'] = (state_race_totals.get('Other', 0) / state_race_totals['Total']) 

# Calculate state and age group totals
state_age_totals = df.groupby(['STATEICP', 'AGE_GROUP'])['PERWT'].sum().unstack(fill_value=0).reset_index()
state_age_totals['Total'] = state_age_totals.sum(axis=1)
age_groups = ['18-24', '25-29', '30-39', '40-49', '50-64', '65+']
for age_group in age_groups:
    state_age_totals[f'{age_group}_Percentage'] = (state_age_totals.get(age_group, 0) / state_age_totals['Total']) 
    
# Calculate state and education level totals
state_education_totals = df.groupby(['STATEICP', 'EDUC'])['PERWT'].sum().unstack(fill_value=0).reset_index()
state_education_totals['Total'] = state_education_totals.sum(axis=1)
education_levels = ['Not a high school graduate', 'High school graduate', 'Some college', 'College graduate', 'Postgraduate education']
for level in education_levels:
    state_education_totals[f'{level.replace(" ", "_")}_Percentage'] = (state_education_totals.get(level, 0) / state_education_totals['Total'])

# Calculate state and income bracket totals
state_income_totals = df.groupby(['STATEICP', 'INCOME_BRACKET'])['PERWT'].sum().unstack(fill_value=0).reset_index()
state_income_totals['Total'] = state_income_totals.sum(axis=1)
income_brackets = ['Missing', 'Under $15,000', '$15,000-30,000', '$30,000-50,000', '$50,000-75,000', '$75,000-100,000', '$100,000-150,000', '$150,000-200,000', 'Over $200,000']
for bracket in income_brackets:
    state_income_totals[f'{bracket.replace(" ", "_").replace("$", "").replace(",", "")}_Percentage'] = (state_income_totals.get(bracket, 0) / state_income_totals['Total']) 

# Calculate state and veteran status totals
state_veteran_totals = df.groupby(['STATEICP', 'VETSTAT'])['PERWT'].sum().unstack(fill_value=0).reset_index()
state_veteran_totals['Total'] = state_veteran_totals.sum(axis=1)
state_veteran_totals['Not_a_veteran_Percentage'] = (state_veteran_totals.get('Not a veteran', 0) / state_veteran_totals['Total']) 
state_veteran_totals['Veteran_Percentage'] = (state_veteran_totals.get('Veteran', 0) / state_veteran_totals['Total']) 

# Filter for White individuals and calculate education levels
df_white = df[df['RACHSING'] == 'White']
df_white['EDUCATION_LEVEL'] = df_white['EDUC'].apply(lambda x: 'College Educated' if x in ['College graduate', 'Postgraduate education'] else 'Non-College Educated')
state_whiteedu_totals = df_white.groupby(['STATEICP', 'EDUCATION_LEVEL'])['PERWT'].sum().unstack(fill_value=0).reset_index()
state_whiteedu_totals['Total'] = state_whiteedu_totals.sum(axis=1)
state_whiteedu_totals['White_Non_College_Percentage'] = (state_whiteedu_totals.get('Non-College Educated', 0) / state_race_totals['Total']) 
state_whiteedu_totals['White_College_Percentage'] = (state_whiteedu_totals.get('College Educated', 0) / state_race_totals['Total']) 

# List of DataFrames to merge
data_frames = [state_sex_totals, state_race_totals, state_age_totals,state_income_totals, state_veteran_totals, state_whiteedu_totals]

# Initialize the merged DataFrame with the first DataFrame
merged_df = data_frames[0]

# Rename or drop the 'Total' column in each DataFrame to avoid conflicts
for i, df in enumerate(data_frames):
    if 'Total' in df.columns:
        df.rename(columns={'Total': f'Total_{i}'}, inplace=True)  # Option 1: Rename
        # df.drop(columns=['Total'], inplace=True)  # Option 2: Drop if not needed

# Iteratively merge the remaining DataFrames
for df in data_frames[1:]:
    merged_df = pd.merge(merged_df, df, on='STATEICP', how='outer')

# Drop columns that are not needed
merged_df = merged_df.loc[:, ~merged_df.columns.str.startswith('Total_')]

# Handle missing values (optional: fill with 0 or leave as NaN based on context)
merged_df = merged_df.fillna(0)

state_mapping = {
    1: "CT",  # Connecticut
    2: "ME",  # Maine
    3: "MA",  # Massachusetts
    4: "NH",  # New Hampshire
    5: "RI",  # Rhode Island
    6: "VT",  # Vermont
    11: "DE", # Delaware
    12: "NJ", # New Jersey
    13: "NY", # New York
    14: "PA", # Pennsylvania
    21: "IL", # Illinois
    22: "IN", # Indiana
    23: "MI", # Michigan
    24: "OH", # Ohio
    25: "WI", # Wisconsin
    31: "IA", # Iowa
    32: "KS", # Kansas
    33: "MN", # Minnesota
    34: "MO", # Missouri
    35: "NE", # Nebraska
    36: "ND", # North Dakota
    37: "SD", # South Dakota
    40: "VA", # Virginia
    41: "AL", # Alabama
    42: "AR", # Arkansas
    43: "FL", # Florida
    44: "GA", # Georgia
    45: "LA", # Louisiana
    46: "MS", # Mississippi
    47: "NC", # North Carolina
    48: "SC", # South Carolina
    49: "TX", # Texas
    51: "KY", # Kentucky
    52: "MD", # Maryland
    53: "OK", # Oklahoma
    54: "TN", # Tennessee
    56: "WV", # West Virginia
    61: "AZ", # Arizona
    62: "CO", # Colorado
    63: "ID", # Idaho
    64: "MT", # Montana
    65: "NV", # Nevada
    66: "NM", # New Mexico
    67: "UT", # Utah
    68: "WY", # Wyoming
    71: "CA", # California
    72: "OR", # Oregon
    73: "WA", # Washington
    81: "AK", # Alaska
    82: "HI", # Hawaii
    98: "DC"  # District of Columbia
}

# Replace STATEICP codes with state names in the DataFrame
merged_df['STATE'] = merged_df['STATEICP'].replace(state_mapping)

# Remove uncessary columns
columns_to_remove = ["STATEICP","Female", "Male", "Asian/Pacific Islander", "Black/African American", "Hispanic/Latino", "Other", "White", "White_Percentage", "18-24"
                     ,"25-29","30-39","40-49","50-64","65+", "$100,000-150,000","$15,000-30,000","$150,000-200,000", "$30,000-50,000","$50,000-75,000","$75,000-100,000",
                     "Missing", "Not a veteran", "Veteran", "College Educated", "Non-College Educated"]
merged_df = merged_df.drop(columns_to_remove, axis=1)

# Export the merged DataFrame to a CSV file
merged_df.to_csv('data/state_demographics.csv', index=False)

print("Data successfully exported to 'state_demographics.csv'")

Column Names: Index(['YEAR', 'MULTYEAR', 'SAMPLE', 'SERIAL', 'CBSERIAL', 'HHWT', 'CLUSTER',
       'STATEICP', 'STRATA', 'GQ', 'PERNUM', 'PERWT', 'SEX', 'AGE', 'RACE',
       'RACED', 'RACHSING', 'EDUC', 'EDUCD', 'FTOTINC', 'VETSTAT', 'VETSTATD'],
      dtype='object')
Missing Values: YEAR              0
MULTYEAR          0
SAMPLE            0
SERIAL            1
CBSERIAL          0
HHWT              0
CLUSTER           0
STATEICP          0
STRATA            0
GQ                0
PERNUM            0
PERWT             0
SEX               0
AGE               0
RACE              0
RACED             0
RACHSING          0
EDUC              0
EDUCD             0
FTOTINC           0
VETSTAT           0
VETSTATD          0
AGE_GROUP         0
INCOME_BRACKET    0
dtype: int64
Unique Values: YEAR                    1
MULTYEAR                3
SAMPLE                  1
SERIAL            3767377
CBSERIAL          3767378
HHWT                  444
CLUSTER           3767378
STATEICP               5

/var/folders/78/ffq4brwd1kq9v6w5l9p1577w0000gn/T/ipykernel_22046/2045259615.py:154: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_white['EDUCATION_LEVEL'] = df_white['EDUC'].apply(lambda x: 'College Educated' if x in ['College graduate', 'Postgraduate education'] else 'Non-College Educated')


Data successfully exported to 'state_demographics.csv'


In [30]:
import pandas as pd

# Load the dataset
df = pd.read_csv('votingagepopulation.csv')

# Ensure 'PERWT' is treated as a numeric column
df['PERWT'] = pd.to_numeric(df['PERWT'], errors='coerce')

# Function to calculate percentages for a single demographic group
def calculate_totals(df, group_col, value_col):
    totals = df.groupby(group_col)[value_col].sum()
    total_weight = totals.sum()
    percentages = (totals / total_weight) 
    return percentages.to_dict()

# Calculate totals and percentages for various demographics
sex_percentages = calculate_totals(df, 'SEX', 'PERWT')
race_percentages = calculate_totals(df, 'RACHSING', 'PERWT')
age_group_percentages = calculate_totals(df, 'AGE_GROUP', 'PERWT')
education_percentages = calculate_totals(df, 'EDUC', 'PERWT')
income_percentages = calculate_totals(df, 'INCOME_BRACKET', 'PERWT')
veteran_percentages = calculate_totals(df, 'VETSTAT', 'PERWT')

# Filter for White individuals and calculate education levels
df_white = df[df['RACHSING'] == 'White']
df_white['EDUCATION_LEVEL'] = df_white['EDUC'].apply(lambda x: 'College Educated' if x in ['College graduate', 'Postgraduate education'] else 'Non-College Educated')
whiteedu_percentages = calculate_totals(df_white, 'EDUCATION_LEVEL','PERWT')
collegeedu = whiteedu_percentages.get('College Educated')
noncollegedu = whiteedu_percentages.get('Non-College Educated')
white_column = race_percentages.get('White')
raceedu_percentages = {
    'College_White': collegeedu*white_column,
    'Noncollege_White': noncollegedu*white_column
}

# Combine all results into a single dictionary
final_results = {
    **sex_percentages,
    **race_percentages,
    **age_group_percentages,
    **education_percentages,
    **income_percentages,
    **veteran_percentages,
    **raceedu_percentages,
}

# Convert to a DataFrame with one row
final_df = pd.DataFrame([final_results])

# Export the final DataFrame to a CSV
final_df.to_csv('national_demographics.csv', index=False)

print("Data successfully exported to 'national_demographics.csv'")


/var/folders/78/ffq4brwd1kq9v6w5l9p1577w0000gn/T/ipykernel_22046/1531929069.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_white['EDUCATION_LEVEL'] = df_white['EDUC'].apply(lambda x: 'College Educated' if x in ['College graduate', 'Postgraduate education'] else 'Non-College Educated')


Data successfully exported to 'national_demographics.csv'
